In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

## Part 1

In [2]:
# Load the demand data by LGAs in 2001
D = pd.read_csv('Demand 2001.csv').set_index(pd.RangeIndex(1, 129, 1))
D

,Local Government Areas (LGAs),Population by LGAs 2001,Babies under 1 year,Children 1-3 years,Children 4-8 years,Adolescents 9-11 years,12-18 years,Women 19-50,Women 51-70,Men 19-70,Adults over 70,Total amount of calcium required by LGAs,Demand by LGAs
1,Albury,45265,566,1841,3173,1944,4416,10473,4292,14583,3976,46952185,43075
2,Armidale,27906,349,1135,1956,1199,2723,6457,2646,8990,2451,28870398,26487
3,Ballina,37856,473,1539,2654,1626,3693,8759,3589,12196,3325,39223163,35985
4,Balranald,2751,34,112,193,118,268,637,261,886,242,2835249,2601
5,Bathurst,35504,444,1444,2489,1525,3464,8215,3366,11438,3119,36773162,33737
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,Wingecarribee,42384,530,1724,2971,1821,4135,9807,4019,13655,3723,43944704,40316
125,Wollondilly,38178,477,1553,2677,1640,3725,8834,3620,12300,3354,39558715,36292
126,Wollongong,188275,2354,7657,13200,8087,18369,43563,17851,60656,16539,199503403,183031
127,Woollahra,52677,659,2142,3693,2263,5139,12188,4994,16971,4627,54701516,50185


In [3]:
# Generate random distances between two LGAs
np.random.seed(0)

# Assign random x and y coordinates to each LGA
coordinates = np.random.randint(0, 1000, (128, 2))

# Calculate the norm between every two LGAs
distance = np.array([[np.linalg.norm(x - y) for y in coordinates] for x in coordinates])
distance = np.round(distance).astype(int)
distance

array([[  0, 371, 254, ..., 398, 207, 533],
       [371,   0, 607, ..., 477, 546, 622],
       [254, 607,   0, ..., 597,  67, 704],
       ...,
       [398, 477, 597, ...,   0, 578, 150],
       [207, 546,  67, ..., 578,   0, 695],
       [533, 622, 704, ..., 150, 695,   0]])

### Discussion
- There used to be more LGAs in 2001, thus we will exclude different LGAs between 2001 and 2021 and assume that there are 128 LGAs in total as in 2021 for consistency across all years.
- Another problem with data is that the age breakdowns of RDI do not match the age breakdowns of NSW population. Specifically, the number of formula fed and breast fed infants are not available, thus we will use the average recommended calcium intake of formula fed babies, breast fed babies and 7-12 months babies, which equals 277 mg/day, as an estimate for the under 1 year age group.
- To generate a reasonable pseudo-dataset for distances between LGAs, we assign random x and y coordinates to each LGA, then calculate the Euclidean distance between two LGAs. These distances will be used as LGA distances as they always satisfy triangle inequality, ensuring that the direct path between two LGAs is the shortest one.

In [4]:
LGA = range(1, 129, 1)
distance = pd.DataFrame(distance, index = LGA, columns = LGA)
demand = pd.Series(D['Demand by LGAs'], index = LGA)

In [5]:
# Define model
mod = gp.Model('Distribution 2001')

# Declare decision variables
x = mod.addVars(LGA, LGA, lb=0, vtype=GRB.CONTINUOUS)
y = mod.addVars(LGA, vtype=GRB.BINARY)

# Add constraints
## Demand
for j in LGA:
    mod.addConstr(sum(x[i, j] for i in LGA) >=  demand[j])
    
## Up to 3 distribution centres
mod.addConstr(y.sum() <= 3)

## Maximum capacity
for i in LGA:
    mod.addConstr(sum(x[i, j] for j in LGA) <= 0.5 * demand.sum())
    
## Linking constraint
M = demand.sum()
for i in LGA:
    for j in LGA:
        mod.addConstr(x[i, j] <= M * y[i])   

# Set objective
distribution_costs = sum(distance.loc[i, j] * x[i, j] * 0.1 * y[i] for i in LGA for j in LGA) 
establishment_costs = sum(y[i] * sum(x[i, j] for j in LGA) * 1000 for i in LGA)

mod.setObjective(distribution_costs + establishment_costs, sense=GRB.MINIMIZE)

mod.update()
mod.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-02
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16641 rows, 16512 columns and 65664 nonzeros
Model fingerprint: 0x2c8022e7
Model has 16384 quadratic objective terms
Variable types: 16384 continuous, 128 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+06]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+06]
Presolve removed 16384 rows and 0 columns
Presolve time: 0.05s
Presolved: 16641 rows, 32896 columns, 82176 nonzeros
Variable types: 32768 continuous, 128 integer (128 binary)
Found heuristic solution: objective 6.481667e+09

Root relaxation: objective 6.295195e+09, 256 iterations, 0.01 seconds (0.0

In [6]:
print(f'The optimal cost is ${round(mod.objVal,0)}')

The optimal cost is $6425329825.0


In [7]:
for i in y:
    if y[i].x > 1e-6:
        print(f"The distribution centre is located in LGA {i}: {D.loc[i, 'Local Government Areas (LGAs)']}")

The distribution centre is located in LGA 10: Blacktown
The distribution centre is located in LGA 65: Lake Macquarie
The distribution centre is located in LGA 91: Parramatta


### Discussion
- I set 2 decision variables:                                                                                                 
    $x_{ij}$: the amount of milk shipped from LGA i to LGA j                                                                                                                                  
    $y_{i}$: a binary variable that is equal to 1 if a distribution centre is located in LGA i and 0 otherwise
- The objective function aims to minimize the total of distribution costs and establishment costs. 
- There are 4 types of constraint. The demand constrains ensure that the total milk supplied to a LGA meet it demand. The second type of constraints restricts the number of distribution centres to be no more than 3. The external risk distribution constraints ensure that no single distribution centre has more production capacity than 50% of total demand. Finally, The last type of constrains are linking constraints.
- The optimal solution is: $y_{10} = 1$, $y_{65} = 1$, $y_{91} = 1$ and all other binary variables are equal to 0, indicating that there are 3 milk distribution centres that are located in Blacktown, Lake Macquarie and Parramatta. The optimal cost is \\$6425329825

    

## Part 2

In [8]:
Demand_2002_2004 = pd.read_csv('Demand 2002 - 2004.csv').set_index(pd.RangeIndex(1, 129, 1))
Demand_2002_2004

,LGAs,Demand 2002,Demand 2003,Demand 2004
1,Albury,47383,52121,57333
2,Armidale,29135,32049,35254
3,Ballina,39583,43541,47895
4,Balranald,2861,3147,3462
5,Bathurst,37111,40822,44904
...,...,...,...,...
124,Wingecarribee,44348,48783,53661
125,Wollondilly,39922,43914,48305
126,Wollongong,201334,221467,243614
127,Woollahra,55203,60724,66796


### Discussion
With an annual population growth of 10%, we can calculate the population in 2002, 2003 and 2004 based population in 2001. Assuming that age breakdowns did not change, the new demand by LGAs can be obtained using the new populations above.

In [9]:
# Rerun the model with data in 2002
d2002 = pd.Series(Demand_2002_2004['Demand 2002'], index = LGA)

In [10]:
# Define model
mod02 = gp.Model('Distribution 2002')

# Declare decision variables
x = mod02.addVars(LGA, LGA, lb=0, vtype=GRB.CONTINUOUS)
y = mod02.addVars(LGA, vtype=GRB.BINARY)

# Add constraints
## Demand
for j in LGA:
    mod02.addConstr(sum(x[i, j] for i in LGA) >=  d2002[j])
    
## Up to 3 distribution centres
mod02.addConstr(y.sum() <= 3)

## Maximum capacity
for i in LGA:
    mod02.addConstr(sum(x[i, j] for j in LGA) <= 0.5 * d2002.sum())
    
## Linking constraint
for i in LGA:
    for j in LGA:
        mod02.addConstr(x[i, j] <= d2002.sum() * y[i])   

# Set objective
distribution_costs = sum(distance.loc[i, j] * x[i, j] * 0.1 for i in LGA for j in LGA) 
establishment_costs = sum(y[i] * sum(x[i, j] for j in LGA) * 1000 for i in LGA)

mod02.setObjective(distribution_costs + establishment_costs, sense=GRB.MINIMIZE)

mod02.update()
mod02.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16641 rows, 16512 columns and 65664 nonzeros
Model fingerprint: 0x3681daad
Model has 16384 quadratic objective terms
Variable types: 16384 continuous, 128 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+06]
  Objective range  [4e-01, 1e+02]
  QObjective range [2e+03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+06]
Presolve removed 16384 rows and 0 columns
Presolve time: 0.03s
Presolved: 16641 rows, 32896 columns, 82176 nonzeros
Variable types: 32768 continuous, 128 integer (128 binary)
Found heuristic solution: objective 7.129840e+09

Root relaxation: objective 6.924721e+09, 256 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Ex

In [11]:
print(f'The optimal cost is ${round(mod02.objVal,0)}')

The optimal cost is $7067869566.0


In [12]:
for i in y:
    if y[i].x > 1e-6:
        print(f"The distribution centre is located in LGA {i}: {D.loc[i, 'Local Government Areas (LGAs)']}")

The distribution centre is located in LGA 10: Blacktown
The distribution centre is located in LGA 65: Lake Macquarie
The distribution centre is located in LGA 91: Parramatta


In [13]:
# Rerun model with data in 2003
d2003 = pd.Series(Demand_2002_2004['Demand 2003'], index = LGA)

In [14]:
# Define model
mod03 = gp.Model('Distribution 2003')

# Declare decision variables
x = mod03.addVars(LGA, LGA, lb=0, vtype=GRB.CONTINUOUS)
y = mod03.addVars(LGA, vtype=GRB.BINARY)

# Add constraints
## Demand
for j in LGA:
    mod03.addConstr(sum(x[i, j] for i in LGA) >=  d2003[j])
    
## Up to 3 distribution centres
mod03.addConstr(y.sum() <= 3)

## Maximum capacity
for i in LGA:
    mod03.addConstr(sum(x[i, j] for j in LGA) <= 0.5 * d2003.sum())
    
## Linking constraint
for i in LGA:
    for j in LGA:
        mod03.addConstr(x[i, j] <= d2003.sum() * y[i])   

# Set objective
distribution_costs = sum(distance.loc[i, j] * x[i, j] * 0.1 for i in LGA for j in LGA) 
establishment_costs = sum(y[i] * sum(x[i, j] for j in LGA) * 1000 for i in LGA)

mod03.setObjective(distribution_costs + establishment_costs, sense=GRB.MINIMIZE)

mod03.update()
mod03.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16641 rows, 16512 columns and 65664 nonzeros
Model fingerprint: 0x88073e7d
Model has 16384 quadratic objective terms
Variable types: 16384 continuous, 128 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+06]
  Objective range  [4e-01, 1e+02]
  QObjective range [2e+03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 4e+06]
Presolve removed 16384 rows and 0 columns
Presolve time: 0.03s
Presolved: 16641 rows, 32896 columns, 82176 nonzeros
Variable types: 32768 continuous, 128 integer (128 binary)
Found heuristic solution: objective 7.842816e+09

Root relaxation: objective 7.617185e+09, 256 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Ex

In [15]:
print(f'The optimal cost is ${round(mod03.objVal,0)}')

The optimal cost is $7774648214.0


In [16]:
for i in y:
    if y[i].x > 1e-6:
        print(f"The distribution centre is located in LGA {i}: {D.loc[i, 'Local Government Areas (LGAs)']}")

The distribution centre is located in LGA 10: Blacktown
The distribution centre is located in LGA 65: Lake Macquarie
The distribution centre is located in LGA 91: Parramatta


In [17]:
# Rerun the model with data in 2004
d2004 = pd.Series(Demand_2002_2004['Demand 2004'], index = LGA)

In [18]:
# Define model
mod04 = gp.Model('Distribution 2004')

# Declare decision variables
x = mod04.addVars(LGA, LGA, lb=0, vtype=GRB.CONTINUOUS)
y = mod04.addVars(LGA, vtype=GRB.BINARY)

# Add constraints
## Demand
for j in LGA:
    mod04.addConstr(sum(x[i, j] for i in LGA) >=  d2004[j])
    
## Up to 3 distribution centres
mod04.addConstr(y.sum() <= 3)

## Maximum capacity
for i in LGA:
    mod04.addConstr(sum(x[i, j] for j in LGA) <= 0.5 * d2004.sum())
    
## Linking constraint
for i in LGA:
    for j in LGA:
        mod04.addConstr(x[i, j] <= d2004.sum() * y[i])   

# Set objective
## Distribution costs
distribution_costs = sum(distance.loc[i, j] * x[i, j] * 0.1 for i in LGA for j in LGA) 
establishment_costs = sum(y[i] * sum(x[i, j] for j in LGA) * 1000 for i in LGA)

mod04.setObjective(distribution_costs + establishment_costs, sense=GRB.MINIMIZE)

mod04.update()
mod04.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16641 rows, 16512 columns and 65664 nonzeros
Model fingerprint: 0xf8cd0b41
Model has 16384 quadratic objective terms
Variable types: 16384 continuous, 128 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+06]
  Objective range  [4e-01, 1e+02]
  QObjective range [2e+03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 4e+06]
Presolve removed 16384 rows and 0 columns
Presolve time: 0.05s
Presolved: 16641 rows, 32896 columns, 82176 nonzeros
Variable types: 32768 continuous, 128 integer (128 binary)
Found heuristic solution: objective 8.627104e+09

Root relaxation: objective 8.378910e+09, 256 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Ex

In [19]:
print(f'The optimal cost is ${round(mod04.objVal,0)}')

The optimal cost is $8552119571.0


In [20]:
for i in y:
    if y[i].x > 1e-6:
        print(f"The distribution centre is located in LGA {i}: {D.loc[i, 'Local Government Areas (LGAs)']}")

The distribution centre is located in LGA 10: Blacktown
The distribution centre is located in LGA 65: Lake Macquarie
The distribution centre is located in LGA 91: Parramatta


### Discussion
- As the population increases, so does the demand for milk, leading to higher distribution costs. Furthermore, the increase in demand might result in potential need for capacity expansions, which raise the setup costs. Therefore, as can be seen from the new optimal solutions, the optimal costs experienced consistent increase to \\$7067869566, \\$7774648214 and \\$8552119571 in 2002, 2003 and 2004 respectively. 
- Despite the increasing costs, the locations for three milk distribution centres remain constant across all years. This suggests that these centres can handle the increasing demand without requiring relocation. The consistency of milk distribution centres is crucial as it indicates that government's intention for each facility to have a lifespan of at least 20 years might be viable. 
- The sensitivity analysis has shown that the government's plan to establish distribution centres in Blacktown, Lake Macquarie and Parramatta is feasible for the growth over the next three years. However, since only assumed changes in population are taken into account in this situation, more in-depth growth forecasts in different aspects will be essential to ensure the lifespans of the distribution centres in the long run.

## Part 3

### Discussion
Beside population growth, there are a number of factors affecting lifespan of the milk distribution centres that need to be considered:
- Developments in transportation infrastructure could reduce the costs of distribution, while deterioration of certain routes can increase distribution costs. Generally, changes in conditions of roads might affect the optimal locations for distribution centres.
- Over two decades, technology advancements including innovations in food preservation, transportation may result in cost savings and improved efficiency.
- Customer preference can be shift over time. There might be more demand for plant-based milk and other sources of calcium, leading to a decrease in demand for milk.
- As research progresses, the recommended dietary intake (RDI) for various nutrients including calcium might be revised. For instance, an increase in the RDI for calcium it might lead to increased milk consumption. Conversely, demand for milk can reduce if the RDI decreases. 
- Inflation can significantly impact the costs of production, costs of distribution and operating costs, thus it is vital to consider the financial risks to manage the economic viability of the milk distribution network over a long-term horizon.
- Another factor needed taking into account is the shifts in age demographics, which could impact demand for milk as some age groups have higher RDI and thus higher demand than others.

In short, it is crucial for the government to conduct regular market research and stay updated with new advancements to ensure the milk distribution strategy remains robust over 20 years.

## Part 4

In [21]:
# Demand data by LGAs in 2006
d2006 = pd.read_csv('Demand 2006.csv').set_index(pd.RangeIndex(1, 129, 1))
d2006

,LGAs,Population 2006,Babies under 1 year,Children 1-3 years,Children 4-8 years,Adolescents 9-11 years,12-18 years,Women 19-50,Women 51-70,Men 19-70,Adults over 70,Calcium recommended,Demand
1,Albury,47566,633,1818,3106,1919,4535,10773,4974,15412,4396,49664257,45564
2,Armidale,27377,364,1046,1788,1105,2610,6200,2863,8870,2530,28486754,26135
3,Ballina,39537,526,1511,2582,1595,3770,8954,4134,12810,3654,41224841,37821
4,Balranald,2507,33,96,164,101,239,568,262,812,232,2597578,2383
5,Bathurst,36916,491,1411,2411,1489,3520,8361,3860,11961,3412,38474810,35298
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,Wingecarribee,43532,579,1664,2843,1756,4151,9859,4552,14105,4023,45421198,41671
125,Wollondilly,40969,545,1566,2675,1653,3906,9279,4284,13274,3786,42728367,39200
126,Wollongong,190909,2541,7298,12467,7703,18202,43237,19962,61856,17644,204178380,187320
127,Woollahra,52995,705,2026,3461,2138,5053,12002,5541,17171,4898,55383713,50811


In [22]:
d2006 = pd.Series(d2006['Demand'], index = LGA)

In [23]:
# Define model
mod06 = gp.Model('Distribution 2006')

# Declare decision variables
x = mod06.addVars(LGA, LGA, lb=0, vtype=GRB.CONTINUOUS)
y = mod06.addVars(LGA, vtype=GRB.BINARY)

# Add constraints
## Demand
for j in LGA:
    mod06.addConstr(sum(x[i, j] for i in LGA) >=  d2006[j])
    
## Up to 3 distribution centres
mod06.addConstr(y.sum() <= 3)

## Maximum capacity
for i in LGA:
    mod06.addConstr(sum(x[i, j] for j in LGA) <= 0.5 * d2006.sum())
    
## Linking constraint
for i in LGA:
    for j in LGA:
        mod06.addConstr(x[i, j] <= d2006.sum() * y[i])   

# Set objective including establishment costs
distribution_costs = sum(distance.loc[i, j] * x[i, j] * 0.1 for i in LGA for j in LGA) 
establishment_costs = sum(y[i] * sum(x[i, j] for j in LGA) * 1000 for i in LGA)

mod06.setObjective(distribution_costs + establishment_costs, sense=GRB.MINIMIZE)

mod06.update()
mod06.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16641 rows, 16512 columns and 65664 nonzeros
Model fingerprint: 0xa28c2265
Model has 16384 quadratic objective terms
Variable types: 16384 continuous, 128 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+06]
  Objective range  [4e-01, 1e+02]
  QObjective range [2e+03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+06]
Presolve removed 16384 rows and 0 columns
Presolve time: 0.05s
Presolved: 16641 rows, 32896 columns, 82176 nonzeros
Variable types: 32768 continuous, 128 integer (128 binary)
Found heuristic solution: objective 6.749305e+09

Root relaxation: objective 6.555956e+09, 256 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Ex

In [24]:
print(f'The optimal cost is ${round(mod06.objVal,0)}')

The optimal cost is $6691136081.0


In [25]:
for i in y:
    if y[i].x > 1e-6:
        print(f"The distribution centre is located in LGA {i}: {D.loc[i, 'Local Government Areas (LGAs)']}")

The distribution centre is located in LGA 10: Blacktown
The distribution centre is located in LGA 65: Lake Macquarie
The distribution centre is located in LGA 91: Parramatta


In [26]:
# Set objective excluding establishment costs
distribution_costs = sum(distance.loc[i, j] * x[i, j] * 0.1 for i in LGA for j in LGA) 
establishment_costs = 0

mod06.setObjective(distribution_costs + establishment_costs, sense=GRB.MINIMIZE)

mod06.update()
mod06.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16641 rows, 16512 columns and 65664 nonzeros
Model fingerprint: 0x60633430
Variable types: 16384 continuous, 128 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+06]
  Objective range  [4e-01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+06]

Loaded MIP start from previous solve with objective 1.3518e+08

Presolve time: 0.13s
Presolved: 16641 rows, 16512 columns, 65792 nonzeros
Variable types: 16384 continuous, 128 integer (128 binary)

Root relaxation: cutoff, 5098 iterations, 0.23 seconds (0.31 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1.3518e+

In [27]:
print(f'The optimal cost is ${round(mod06.objVal,0)}')

The optimal cost is $135180081.0


In [28]:
for i in y:
    if y[i].x > 1e-6:
        print(f"The distribution centre is located in LGA {i}: {D.loc[i, 'Local Government Areas (LGAs)']}")

The distribution centre is located in LGA 10: Blacktown
The distribution centre is located in LGA 65: Lake Macquarie
The distribution centre is located in LGA 91: Parramatta


### Discussion
- By 2006, the optimal cost is considerably lower than the 2004 projected cost, \\$6691136081 compared to \\$8552119571 even though two additional years have passed, the expected cost in 2006 is even lower than 2002 and 2003 ones. The reason for this is that the 2006 model use the actual data of Australian Bureau Of Statistics (ABS) while in part 2 there was an assumption that annual population growth is 10%. This result shows that the real growth rate during this period was less than the assumed 10%, which is consistent with research from ABS suggesting a 2006 growth rate of 0.9% in NSW (Australian Bureau Of Statistics, 2007). The locations of distribution centres remain constant considering the 2006 population.
-  The model outcomes seems highly sensitive to the input parameters. In this case, the assumption of a 10% annual growth was considerably higher than the actual growth rate of 0.9% in 2006, thus the precisions of inputs are of great importance to achieve reliable result.
- By excluding establishment costs from the 2006 model, we are considering a scenario where the initial investment associated with setting up the distribution centres have already been integrated in 2001 model. Without the establishment costs, the model primarily focuses on operational efficiency. The optimal cost is much lower while the location choices are stable.

## Part 5

In [29]:
# Demand data by LGAs in 2021 
d2021 = pd.read_csv('Demand 2021.csv').set_index(pd.RangeIndex(1, 129, 1))
d2021

,LGAs,Population 2021,Babies under 1 year,Children 1-3 years,Children 4-8 years,Adolescents 9-11 years,12-18 years,Women 19-50,Women 51-70,Men 19-70,Adults over 70,Calcium recommended,Demand
1,Albury,56036,635,1947,3436,2109,4639,12088,6587,18137,6459,59105590,54225
2,Armidale,29332,333,1019,1798,1104,2428,6327,3448,9494,3381,30838112,28292
3,Ballina,46172,523,1604,2831,1738,3822,9960,5428,14944,5322,48642721,44626
4,Balranald,2207,25,77,135,83,183,476,259,714,254,2312630,2122
5,Bathurst,43653,495,1517,2676,1643,3613,9416,5131,14129,5032,45974794,42179
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,Wingecarribee,52456,595,1822,3216,1975,4342,11315,6166,16978,6046,55305348,50739
125,Wollondilly,54176,614,1882,3322,2039,4485,11686,6368,17535,6245,57130751,52414
126,Wollongong,214657,2433,7458,13161,8081,17769,46304,25233,69476,24743,230791454,211735
127,Woollahra,53891,611,1872,3304,2029,4461,11625,6335,17442,6212,56828234,52136


In [30]:
d2021 = pd.Series(d2021['Demand'], index = LGA)

In [31]:
# Define model
mod21 = gp.Model('Distribution 2021')

# Declare decision variables
x = mod21.addVars(LGA, LGA, lb=0, vtype=GRB.CONTINUOUS)
y = mod21.addVars(LGA, vtype=GRB.BINARY)

# Add constraints
## Demand
for j in LGA:
    mod21.addConstr(sum(x[i, j] for i in LGA) >=  d2021[j])
    
## Up to 3 distribution centres
mod21.addConstr(y.sum() <= 3)

## Maximum capacity
for i in LGA:
    mod21.addConstr(sum(x[i, j] for j in LGA) <= 0.5 * d2021.sum())
    
## Linking constraint
for i in LGA:
    for j in LGA:
        mod21.addConstr(x[i, j] <= d2021.sum() * y[i])   

# Set objective including establishment costs
## Distribution costs
shipping_cost = 0.1 * (1+0.556)
distribution_costs = sum(distance.loc[i, j] * x[i, j] * shipping_cost for i in LGA for j in LGA)

## Establishment costs
setup_cost = 1000 * (1+0.556)
establishment_costs = sum(y[i] * sum(x[i, j] for j in LGA) * setup_cost for i in LGA)
mod21.setObjective(distribution_costs + establishment_costs, sense=GRB.MINIMIZE)

mod21.update()
mod21.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16641 rows, 16512 columns and 65664 nonzeros
Model fingerprint: 0x8dc56455
Model has 16384 quadratic objective terms
Variable types: 16384 continuous, 128 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+06]
  Objective range  [6e-01, 2e+02]
  QObjective range [3e+03, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 4e+06]
Presolve removed 16384 rows and 0 columns
Presolve time: 0.05s
Presolved: 16641 rows, 32896 columns, 82176 nonzeros
Variable types: 32768 continuous, 128 integer (128 binary)
Found heuristic solution: objective 1.271993e+10

Root relaxation: objective 1.235690e+10, 256 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Ex

In [32]:
print(f'The optimal cost is ${round(mod21.objVal,0)}')

The optimal cost is $12607074853.0


In [33]:
for i in y:
    if y[i].x > 1e-6:
        print(f"The distribution centre is located in LGA {i}: {D.loc[i, 'Local Government Areas (LGAs)']}")

The distribution centre is located in LGA 10: Blacktown
The distribution centre is located in LGA 65: Lake Macquarie
The distribution centre is located in LGA 91: Parramatta


In [34]:
# Set objective excluding establishment costs
## Distribution costs
shipping_cost = 0.1 * (1+0.556)
distribution_costs = sum(distance.loc[i, j] * x[i, j] * shipping_cost for i in LGA for j in LGA)

## Establishment costs
establishment_costs = 0

mod21.setObjective(distribution_costs + establishment_costs, sense=GRB.MINIMIZE)

mod21.update()
mod21.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16641 rows, 16512 columns and 65664 nonzeros
Model fingerprint: 0x4c2543aa
Variable types: 16384 continuous, 128 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+06]
  Objective range  [6e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 4e+06]

Loaded MIP start from previous solve with objective 2.50174e+08

Presolve time: 0.11s
Presolved: 16641 rows, 16512 columns, 65792 nonzeros
Variable types: 16384 continuous, 128 integer (128 binary)

Root relaxation: cutoff, 4845 iterations, 0.22 seconds (0.25 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      2.5017e

In [35]:
print(f'The optimal cost is ${round(mod21.objVal,0)}')

The optimal cost is $250173985.0


In [36]:
for i in y:
    if y[i].x > 1e-6:
        print(f"The distribution centre is located in LGA {i}: {D.loc[i, 'Local Government Areas (LGAs)']}")

The distribution centre is located in LGA 10: Blacktown
The distribution centre is located in LGA 65: Lake Macquarie
The distribution centre is located in LGA 91: Parramatta


### Discussion
The combination of 55.6% inflation over two decades and natural population growth has considerably increased the costs. Despite the substantial rise in costs, the locations for distribution centres remain the same both when including and excluding establishment costs. This suggests that the government's plan is viable, whereas distribution centres might need relocating if we use the assumed growth rate of 10%, which is considerably higher than real-world growth rate of 0.1% in 2021 (ABS, 2022), as in part 2. However, the marked increase in optimal costs emphasizes the importance of considering various factors that might impact milk distribution network aside from population growth when planning for the long term.

### References
    1. Australian Bureau Of Statistics. (2001). Usual Residency Profile. https://www.abs.gov.au/census/find-census-data/community-profiles/2001/1
    2. Australian Bureau Of Statistics. (2006). Basic Community Profile. Www.abs.gov.au. https://www.abs.gov.au/census/find-census-data/community-profiles/2006/1
    3. Australian Bureau Of Statistics. (2007, June 5). Australian Demographic Statistics. Abs.gov.au; c=AU; o=Commonwealth of Australia; ou=Australian Bureau of Statistics. https://www.abs.gov.au/AUSSTATS/abs@.nsf/Lookup/3101.0Main+Features1Dec%202006?OpenDocument=
    4. Australian Bureau Of Statistics. (2021). General Community Profile. Www.abs.gov.au. https://www.abs.gov.au/census/find-census-data/community-profiles/2021/1
    5. Australian Bureau of Statistics. (2022, June 28). National, state and territory population, December 2021 . Www.abs.gov.au. https://www.abs.gov.au/statistics/people/population/national-state-and-territory-population/dec-2021
    6. Australian Bureau of Statistics. (2023). Population estimates by LGA, Significant Urban Area, Remoteness Area and electoral division, 2001 to 2022. https://www.abs.gov.au/statistics/people/population/regional-population/latest-release#data-downloads